In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_dataset_search = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
indexing_dataset_search.indices.close(index='dataset_search')
indexing_dataset_search.indices.put_settings(index='dataset_search', body={"index": {"similarity": {"default": {"type": "BM25"}}}})
indexing_dataset_search.indices.open(index='dataset_search')

In [ ]:
def elasticsearch_search(query):
    
    result= indexing_dataset_search.search(
        index="dataset_search", 
        body = {
        "_source": ["dataset_id","dataset_title","dataset_description","dataset_tags"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["dataset_description"] 
            }
        }
    })
    
    return result

In [ ]:
train_queries = pd.read_csv('../data/final_train_queries.csv')

In [ ]:
train_queries.info()

In [ ]:
top_100_index = []

for i,row in tqdm(train_queries.iterrows()):

    query_id = row['query_id']
    query_description = str(row['query_description'])

    result = elasticsearch_search(query_description)
    
    return_datasets = []

    for hit in result['hits']['hits']:

        dataset_id = hit['_source']['dataset_id']
        dataset_title = hit['_source']['dataset_title']
        dataset_description = hit['_source']['dataset_description']
        dataset_tags = hit['_source']['dataset_tags']

        line = {"label_index":query_id,
                "dataset_id":dataset_id,
                "dataset_title":dataset_title,
                "dataset_description":dataset_description,
                "dataset_tags":dataset_tags}

        top_100_index.append(line)

In [ ]:
df_top100 = pd.DataFrame(top_100_index)

In [ ]:
df_top100.to_csv('fixed_test_set_index_top100.csv',index=False)